In [0]:
!wget https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
!unzip -qq cats_and_dogs_filtered.zip

In [0]:
import keras
from keras.models import Sequential 
from keras.layers import Conv2D, MaxPooling2D , BatchNormalization, GlobalAveragePooling2D
from keras.layers import Activation, Dropout, Flatten, Dense 
from keras import backend as K 

def MODEL(img_width, img_height, num_classes):
    if K.image_data_format() == 'channels_first': 
        input_shape = (3, img_width, img_height) 
    else: 
        input_shape = (img_width, img_height, 3) 
  
    model = Sequential() 
    model.add(Conv2D(32, (3, 2), input_shape = input_shape)) 
    model.add(BatchNormalization())
    model.add(Activation('relu')) 
    
    model.add(Conv2D(64, (3, 2)))
    model.add(BatchNormalization())
    model.add(Activation('relu')) 
    
    model.add(Conv2D(128, (3, 2)))
    model.add(BatchNormalization())
    model.add(Activation('relu')) 

    model.add(GlobalAveragePooling2D()) 
    
    model.add(Dense(64)) 
    model.add(Activation('relu')) 
    model.add(Dropout(0.5)) 
    model.add(Dense(num_classes)) 
    model.add(Activation('sigmoid')) 
    return model

In [0]:
import os
import tensorflow as tf
import keras
import json
from keras.preprocessing.image import ImageDataGenerator 
from keras import optimizers

def main():
    img_width, img_height = 224, 224
    EPOCHS     = 50
    BATCH_SIZE = 16

    train_datagen = ImageDataGenerator( rescale = 1. / 255, rotation_range = 15) 
    test_datagen = ImageDataGenerator( rescale = 1. / 255 )

    train_data_dir = '/content/cats_and_dogs_filtered/train'
    validation_data_dir = '/content/cats_and_dogs_filtered/validation'

    label_list = os.listdir(train_data_dir)
    label_list.sort() 

    train_generator = train_datagen.flow_from_directory(
        train_data_dir
        , target_size = (img_width, img_height)
        , shuffle = True
        , batch_size = BATCH_SIZE
        , classes = label_list
        , class_mode ='categorical') 

    validation_generator = test_datagen.flow_from_directory( 
        validation_data_dir
        , target_size = (img_width, img_height)
        , shuffle = True
        , batch_size = BATCH_SIZE
        , classes = label_list
        , class_mode ='categorical') 

    f = open("label_map.txt", 'w')
    for i in range(len(label_list)):
        f.write(label_list[i]+'\n')
    f.close()    
    num_classes = len(label_list)

    model = MODEL(img_width, img_height, num_classes)
    model_str = "keras_Classification_Model"

    optimizer = optimizers.Adam(lr=0.0001)
    model.compile(optimizer = optimizer
                , loss = 'categorical_crossentropy'
                , metrics = ['accuracy'])


    model.fit_generator(generator = train_generator
                        , steps_per_epoch = train_generator.n//train_generator.batch_size
                        , epochs = EPOCHS
                        , validation_data = validation_generator
                        , validation_steps = validation_generator.n//validation_generator.batch_size)


    model_json = model.to_json()
    with open(model_str + ".json", "w") as json_file:
        json.dump(model_json, json_file)
    model.save_weights(model_str + ".h5") 

In [0]:
main()